## Scraping multiple pages in Carpages

In [1]:
import time
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
url = 'https://www.carpages.ca/used-cars/search/?num_results=50&fueltype_id%5b0%5d=3&fueltype_id%5b1%5d=7&p=3'

In [3]:
page =requests.get (url, headers = {'User-agent': 'your bot 0.1'})
page

<Response [200]>

In [4]:
soup = BeautifulSoup(page.text, 'lxml')

In [5]:
#Creating our dataframe
df = pd.DataFrame({'Link':[''], 'Name':[''], 'Price':[''], 'Color':['']})
df

,Link,Name,Price,Color
0,,,,


In [6]:
%%time

counter = 0
#This loop goes through the first 10 pages and grabs all the details of each posting
while counter < 5:
    
    #gets the HTML of all the postings on the page
    postings = soup.find_all('div', class_ = 'media soft push-none rule')

    #grabs all the details for each posting and adds it as a row to the dataframe
    for post in postings:
        link = post.find('a', class_ = 'media__img media__img--thumb').get('href')
        link_full = 'https://www.carpages.ca' + link
        name = post.find('h4', class_ = 'hN').text.strip()
        price = post.find('strong', class_ = 'delta').text.strip()
        color = post.find_all('div', class_ = 'grey l-column l-column--small-6 l-column--medium-4')[1].text.strip()
        df = pd.concat([df, pd.DataFrame.from_records([{'Link':link_full, 'Name':name,'Price':price, 'Color':color}])], ignore_index=True)
    
    try:
        #grabs the url of the next page
        next_page = soup.find('a', class_ = 'nextprev').get('href')
        
        #Imports the next pages HTML into python        
        page = 'https://www.carpages.ca' + next_page
        soup = BeautifulSoup(page.text, 'lxml')
        counter += 1
        time.sleep(20)
    except AttributeError:
        break
    



CPU times: total: 62.5 ms
Wall time: 50.9 ms


In [7]:
df.head()

,Link,Name,Price,Color
0,,,,
1,https://www.carpages.ca/used-cars/ontario/miss...,2018 Ford F-250,"$59,998",White Platinum Tri-Coat Metallic
2,https://www.carpages.ca/used-cars/ontario/miss...,2019 Ford F-250,"$53,998",White Platinum Tri-Coat Metallic
3,https://www.carpages.ca/used-cars/ontario/miss...,2020 Ford Explorer,"$54,998",Magnetic Metallic
4,https://www.carpages.ca/used-cars/ontario/miss...,2019 Ford F-150,"$38,998",Agate Black


In [8]:
# Remove 1st row
df = df.iloc[1: , :]
df.head()

,Link,Name,Price,Color
1,https://www.carpages.ca/used-cars/ontario/miss...,2018 Ford F-250,"$59,998",White Platinum Tri-Coat Metallic
2,https://www.carpages.ca/used-cars/ontario/miss...,2019 Ford F-250,"$53,998",White Platinum Tri-Coat Metallic
3,https://www.carpages.ca/used-cars/ontario/miss...,2020 Ford Explorer,"$54,998",Magnetic Metallic
4,https://www.carpages.ca/used-cars/ontario/miss...,2019 Ford F-150,"$38,998",Agate Black
5,https://www.carpages.ca/used-cars/ontario/miss...,2016 Ford F-150,"$34,998",Lithium Grey


In [9]:
#exports the data as a csv
df.to_csv('CarPages.csv', index=False)